In [10]:
from models.PCA import PCA
from models.FF import FF

import pandas as pd
import numpy as np
from tqdm import tqdm
from utils import charas
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
def model_inference_and_predict(model):
    mon_list = pd.read_pickle('data/mon_list.pkl')
    test_mons = mon_list.loc[mon_list >= model.test_period[0]]
    inference_result = []
    predict_result = []
    T_bar = tqdm(test_mons.groupby(test_mons.apply(lambda x: x//10000)), colour='red', desc=f'{model.name} Inferencing & Predicting')
    
    for g in T_bar: # rolling train
        T_bar.set_postfix({'Year': g[0]})
        model.train_model()
        
        for m in g[1].to_list():
            inference_result.append(model.inference(m))
            predict_result.append(model.predict(m))
        # model refit (change train period and valid period)
        model.refit()

    inference_result = pd.DataFrame(inference_result, index=test_mons, columns=charas)
    inference_result.to_csv(f'results/inference/{model.name}_inference.csv')
    
    predict_result = pd.DataFrame(predict_result, index=test_mons, columns=charas)
    predict_result.to_csv(f'results/predict/{model.name}_predict.csv')

In [ ]:
pca_1 = PCA(K=1, portfolio=True)
pca_2 = PCA(K=2, portfolio=True)
pca_3 = PCA(K=3, portfolio=True)
pca_4 = PCA(K=4, portfolio=True)
pca_5 = PCA(K=5, portfolio=True)
pca_6 = PCA(K=6, portfolio=True)

In [ ]:
model_inference_and_predict(pca_1)
# model_inference_and_predict(pca_2)
# model_inference_and_predict(pca_3)
# model_inference_and_predict(pca_4)
# model_inference_and_predict(pca_5)
# model_inference_and_predict(pca_6)

In [5]:
ff_1 = FF(K=1, portfolio=True)
ff_2 = FF(K=2, portfolio=True)
ff_3 = FF(K=3, portfolio=True)
ff_4 = FF(K=4, portfolio=True)
ff_5 = FF(K=5, portfolio=True)
ff_6 = FF(K=6, portfolio=True)

In [6]:
model_inference_and_predict(ff_1)
model_inference_and_predict(ff_2)
model_inference_and_predict(ff_3)
model_inference_and_predict(ff_4)
model_inference_and_predict(ff_5)
model_inference_and_predict(ff_6)

FF_6 Inferencing & Predicting: 100%|██████████| 30/30 [00:06<00:00,  4.75it/s, Year=2016]


In [19]:
def model_inference_and_predict_CA(model):
    mon_list = pd.read_pickle('data/mon_list.pkl')
    test_mons = mon_list.loc[mon_list >= model.test_period[0]]
    inference_result = pd.DataFrame()
    predict_result = pd.DataFrame()
    T_bar = tqdm(test_mons.groupby(test_mons.apply(lambda x: x//10000)), colour='red', desc=f'{model.name} Inferencing & Predicting')
    
    stock_index = pd.Series(dtype=np.int64)
    for g in T_bar: # rolling train
        T_bar.set_postfix({'Year': g[0]})
        model.train_model()
        
        for m in g[1].to_list():
            m_stock_index, _, _, _ = model._get_item(m)
            print(m_stock_index)
            stock_index = pd.concat([stock_index, pd.Series(m_stock_index)]).drop_duplicates().astype(int)
            inference_R = model.inference(m) # return (N, 1)
            predict_R = model.inference(m) # reutrn (N, 1)

            # move inference_R and predict_R to cpu
            inference_R = inference_R.cpu().detach().numpy()
            predict_R = predict_R.cpu().detach().numpy()

            inference_R = pd.DataFrame(inference_R, index=m_stock_index, columns=[m]).reindex(stock_index)
            predict_R = pd.DataFrame(predict_R, index=m_stock_index, columns=[m]).reindex(stock_index)

            
            inference_result = pd.concat([inference_result, inference_R], axis=1) # (N, T)
            predict_result = pd.concat([predict_result, predict_R], axis=1) # (N, T)
        # model refit (change train period and valid period)
        model.refit()

    inference_result = pd.DataFrame(inference_result, index=test_mons, columns=charas)
    inference_result.to_csv(f'results/inference/{model.name}_inference_stock.csv')
    
    predict_result = pd.DataFrame(predict_result, index=test_mons, columns=charas)
    predict_result.to_csv(f'results/predict/{model.name}_predict_stock.csv')

In [12]:
from models.CA import CA0, CA1, CA2, CA3
import gc

In [20]:
ca3_lists = []
for i in range(6):
    gc.collect()
    ca3_lists.append(CA3(i + 1).to('cuda'))
    print(f'begin of {ca3_lists[i].name}')
    model_inference_and_predict_CA(ca3_lists[i])


begin of CA3


CA3 Inferencing & Predicting:   0%|          | 0/30 [00:00<?, ?it/s, Year=1987]

Index([10000.0, 10001.0, 10002.0, 10003.0, 10005.0, 10007.0, 10008.0, 10009.0,
       10010.0, 10012.0,
       ...
       93121.0, 93156.0, 93164.0, 93201.0, 93220.0, 93236.0, 93252.0, 93279.0,
       93287.0, 93316.0],
      dtype='float64', name='permno', length=6600)


CA3 Inferencing & Predicting:   0%|          | 0/30 [00:30<?, ?it/s, Year=1987]


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.